In [10]:
import pandas as pd

# Load the datasets
survey_data = pd.read_csv('adjusted_reponse_survey.csv')
contract_data = pd.read_csv('Notebooks\experiment\curr_data.csv')

from scipy.stats import wilcoxon, shapiro

# Merge the datasets on player names for analysis
merged_data = contract_data.merge(
    survey_data, how='left', left_on='Player ID', right_on='PLAYER NAME'
)

contract_data.head()


<>:5: SyntaxWarning: invalid escape sequence '\e'
<>:5: SyntaxWarning: invalid escape sequence '\e'
C:\Users\telat\AppData\Local\Temp\ipykernel_59028\839069531.py:5: SyntaxWarning: invalid escape sequence '\e'
  contract_data = pd.read_csv('Notebooks\experiment\curr_data.csv')


,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,...,Expected_Leftovers,Expected Retailer Profit,Expected Mfg Profit,Expected Mfg. Profit Share,Predicted Sales,Predicted Leftovers,Predicted Retailer Profit,Predicted Mfg Profit,Realized Retailer Profit,Predicted Mfg. Profit Share
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,...,23.45,234.6,300,0.561167,77.72,5.28,268.64,415,240,0.607045
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,...,15.35,347.8,320,0.479185,82.80,8.20,356.60,364,400,0.505135
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,...,15.35,347.8,320,0.479185,82.80,8.20,356.60,364,124,0.505135
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,...,24.00,153.0,306,0.666667,72.00,3.00,189.00,450,153,0.704225
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,...,23.45,234.6,300,0.561167,77.72,5.28,268.64,415,240,0.607045


In [11]:
# Standardize identifiers in both datasets to facilitate merging

# Convert 'Player ID' in contract_data to lowercase and remove spaces to match survey_data
contract_data['Player ID'] = contract_data['Player ID'].str.replace(" ", "").str.lower()

# Re-attempt merging on the standardized 'Player ID' and 'PLAYER NAME' columns
merged_data = contract_data.merge(
    survey_data, how='left', left_on='Player ID', right_on='PLAYER NAME'
)

# Check if the Risk Averse Coefficient values are now successfully transferred in the merged dataset
merged_data['Risk Averse Coefficient'].isna().sum(), merged_data['Risk Averse Coefficient'].describe()


merged_data.head()



KeyError: 'Risk Averse Coefficient'

In [6]:

# Hypothesis 1: Does Risk Aversion affect the difference between actual and optimal stock levels?
# Null Hypothesis (H0): There is no significant difference in the stock-keeping behavior of high-risk-averse and low-risk-averse individuals.
# Alternative Hypothesis (H1): There is a significant difference in the stock-keeping behavior based on risk aversion.

from scipy.stats import mannwhitneyu, shapiro

# Function to get the top and bottom 4 based on a given column
def get_top_bottom_4(data, column):
    sorted_data = data.sort_values(by=column)
    top_4 = sorted_data.tail(4)
    bottom_4 = sorted_data.head(4)
    return top_4, bottom_4

# Example for Risk Aversion and Stock Level Difference
top_risk_averse, bottom_risk_averse = get_top_bottom_4(merged_data, 'Risk Averse Coefficient')

# Calculate the difference between realized and optimal stock for both groups
top_risk_diff = top_risk_averse['Stock'] - top_risk_averse['Optimal_Stock']
bottom_risk_diff = bottom_risk_averse['Stock'] - bottom_risk_averse['Optimal_Stock']

# Conduct Mann-Whitney U test
test_stat, p_value = mannwhitneyu(top_risk_diff.dropna(), bottom_risk_diff.dropna(), alternative='less')
print("Risk Aversion Hypothesis Test:", test_stat, p_value)

# Determine the direction of the difference
mean_top_risk = top_risk_diff.mean()
mean_bottom_risk = bottom_risk_diff.mean()

print(f"Mean Stock Difference - Top Risk Averse: {mean_top_risk:.2f}")
print(f"Mean Stock Difference - Bottom Risk Averse: {mean_bottom_risk:.2f}")

if mean_top_risk > mean_bottom_risk:
    print("The top 4 risk-averse individuals tend to overorder compared to the bottom 4.")
elif mean_top_risk < mean_bottom_risk:
    print("The top 4 risk-averse individuals tend to underorder compared to the bottom 4.")
else:
    print("No directional difference observed between the top and bottom risk-averse groups.")




Risk Aversion Hypothesis Test: 3.0 0.1
Mean Stock Difference - Top Risk Averse: -33.25
Mean Stock Difference - Bottom Risk Averse: -4.75
The top 4 risk-averse individuals tend to underorder compared to the bottom 4.


In [15]:
# Conduct Wilcoxon test (since normality might not hold)
_, high_risk_pval = shapiro(high_risk_diff.dropna())
_, low_risk_pval = shapiro(low_risk_diff.dropna())

# Re-attempting with the Mann-Whitney U test for independent samples with unequal sizes
from scipy.stats import mannwhitneyu

# Conduct Mann-Whitney U test as an alternative to Wilcoxon due to unequal sample sizes
test_stat, p_value = mannwhitneyu(high_risk_diff.dropna(), low_risk_diff.dropna(), alternative='two-sided')

# (98215.5, 0.555089882203615)
test_stat, p_value

"""
The Mann-Whitney U test results in a p-value of approximately 0.555, indicating no statistically significant difference between high-risk-averse and low-risk-averse groups 
in terms of their deviations from the optimal stock level. This suggests that risk aversion may not significantly impact stocking decisions in this experiment.
"""




(98215.5, 0.555089882203615)

In [16]:
# Hypothesis Test: Impact of Self-Esteem on Deviation from Optimal Stock Levels
# Null Hypothesis (H0): There is no significant difference in the deviation from optimal stock level between high and low self-esteem groups.
# Alternative Hypothesis (H1): There is a significant difference in deviation based on self-esteem.

# Median split for self-esteem
esteem_median = merged_data['Self Esteem Average'].median()
high_esteem = merged_data[merged_data['Self Esteem Average'] >= esteem_median]
low_esteem = merged_data[merged_data['Self Esteem Average'] < esteem_median]

# Calculate stock deviation from optimal stock for both groups
high_esteem_diff = high_esteem['Stock'] - high_esteem['Optimal_Stock']
low_esteem_diff = low_esteem['Stock'] - low_esteem['Optimal_Stock']

# Conduct Mann-Whitney U test to account for unequal sample sizes
test_stat, p_value = mannwhitneyu(high_esteem_diff.dropna(), low_esteem_diff.dropna(), alternative='two-sided')

test_stat, p_value

"""
The Mann-Whitney U test yields a p-value of approximately 0.0036, indicating a statistically significant 
difference in deviations from optimal stock levels between high and low self-esteem groups. 
This suggests that self-esteem may indeed influence ordering behavior, with higher 
self-esteem potentially aligning more closely with optimal stock levels.
"""


(107778.5, 0.003582084997420401)

In [17]:
# Hypothesis Test: Influence of Fairness Index on Wholesale Price Acceptance
# Null Hypothesis (H0): There is no significant difference in the reaction to wholesale price changes based on fairness index.
# Alternative Hypothesis (H1): There is a significant difference in wholesale price acceptance based on fairness.

# Median split for fairness index
fairness_median = merged_data['Fairness Index'].median()
high_fairness = merged_data[merged_data['Fairness Index'] >= fairness_median]
low_fairness = merged_data[merged_data['Fairness Index'] < fairness_median]

# Calculate wholesale price for both groups
high_fairness_wholesale = high_fairness['Wholesale p.']
low_fairness_wholesale = low_fairness['Wholesale p.']

# Conduct Mann-Whitney U test for wholesale price acceptance in both groups
test_stat, p_value = mannwhitneyu(high_fairness_wholesale.dropna(), low_fairness_wholesale.dropna(), alternative='two-sided')

test_stat, p_value

"""
The Mann-Whitney U test results in a p-value of approximately 0.0002, indicating a statistically significant difference 
in reactions to wholesale price changes based on fairness index. This suggests that individuals with higher fairness indexes
 may respond differently to price fluctuations compared to those with lower fairness indexes, potentially being more 
 resistant to or influenced by changes in wholesale prices.
"""


(109357.0, 0.00019240731573129556)

In [19]:
# Hypothesis Test: Regret Sensitivity and Over/Under-Stocking Behavior
# Null Hypothesis (H0): There is no significant difference in over/under-stocking behavior based on regret sensitivity.
# Alternative Hypothesis (H1): There is a significant difference in over/under-stocking based on regret sensitivity.

# Median split for Regret Scale
regret_median = merged_data['Regret Scale Average'].median()
high_regret = merged_data[merged_data['Regret Scale Average'] >= regret_median]
low_regret = merged_data[merged_data['Regret Scale Average'] < regret_median]

# Calculate over/under-stocking for both groups
high_regret_diff = high_regret['Stock'] - high_regret['Optimal_Stock']
low_regret_diff = low_regret['Stock'] - low_regret['Optimal_Stock']

# Conduct Mann-Whitney U test for over/under-stocking behavior based on regret sensitivity
test_stat, p_value = mannwhitneyu(high_regret_diff.dropna(), low_regret_diff.dropna(), alternative='two-sided')

test_stat, p_value
"""
The Mann-Whitney U test yields a p-value of approximately 0.178, indicating no statistically significant difference
 in over- or under-stocking behavior between high and low regret-sensitive participants. 
This suggests that regret sensitivity may not have a strong impact on deviations from the optimal stock level.
"""



(91717.5, 0.1775270055865822)

In [21]:
# Hypothesis Test: Fairness Index and Retailer Profit Share
# Null Hypothesis (H0): There is no significant difference in retailer profit share based on fairness index.
# Alternative Hypothesis (H1): There is a significant difference in retailer profit share based on fairness index.

# Median split for fairness index
fairness_median = merged_data['Fairness Index'].median()
high_fairness = merged_data[merged_data['Fairness Index'] >= fairness_median]
low_fairness = merged_data[merged_data['Fairness Index'] < fairness_median]

# Calculate realized retailer profit share for both groups
high_fairness_profit_share = high_fairness['Realized Retailer Profit'] / (
    high_fairness['Realized Retailer Profit'] + high_fairness['Realized_Mfg_Profit'])
low_fairness_profit_share = low_fairness['Realized Retailer Profit'] / (
    low_fairness['Realized Retailer Profit'] + low_fairness['Realized_Mfg_Profit'])

# Conduct Mann-Whitney U test for retailer profit share based on fairness index
test_stat, p_value = mannwhitneyu(high_fairness_profit_share.dropna(), low_fairness_profit_share.dropna(), alternative='two-sided')

test_stat, p_value


"""
The Mann-Whitney U test gives a p-value of approximately 0.011, indicating a statistically significant difference in retailer profit share between participants with high and low fairness indexes. 
This suggests that fairness may indeed influence the distribution of profits between the retailer and manufacturer, with higher fairness potentially leading to a more balanced profit share.
"""


(71593.5, 0.011093701206110461)

In [22]:
# Hypothesis Test: Risk Aversion and Demand Responsiveness
# Null Hypothesis (H0): There is no significant difference in demand responsiveness based on risk aversion.
# Alternative Hypothesis (H1): There is a significant difference in demand responsiveness based on risk aversion.

# Calculate responsiveness to demand by taking the absolute difference between demand and stock for each group
high_risk_demand_response = abs(high_risk_averse['Demand'] - high_risk_averse['Stock'])
low_risk_demand_response = abs(low_risk_averse['Demand'] - low_risk_averse['Stock'])

# Conduct Mann-Whitney U test for demand responsiveness based on risk aversion
test_stat, p_value = mannwhitneyu(high_risk_demand_response.dropna(), low_risk_demand_response.dropna(), alternative='two-sided')

test_stat, p_value

"""
The Mann-Whitney U test yields a p-value of approximately 0.086, which does not indicate a statistically significant difference in demand responsiveness between 
high and low risk-averse participants. This suggests that risk aversion may not strongly impact how participants adjust their stock in response to demand changes.
"""


(89551.0, 0.08584266993168418)